<a href="https://colab.research.google.com/github/sumandutta8877/HP3Lab/blob/master/HP3Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [2]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-9t7tcv72
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-9t7tcv72
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=fcd6bdd34306e244bf882f7e998d00341e14747b6c4deca9e55ebe56a26d5bdc
  Stored in directory: /tmp/pip-ephem-wheel-cache-a76kxmrv/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name helloCUDA.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void vectorAdd(const float *A, const float *B, float *C, int numElements)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;

    if (i < numElements)
    {
        C[i] = A[i] + B[i];
    }
}

int main(void)
{
    // Error code to check return values for CUDA calls
    cudaError_t err = cudaSuccess;

    // Print the vector length to be used, and compute its size
    int numElements = 0;
    scanf("%d",&numElements);
    
    size_t size = numElements * sizeof(float);
    printf("[Vector addition of %d elements]\n", numElements);

    // Allocate the host input vector A
    float *h_A = (float *)malloc(size);

    // Allocate the host input vector B
    float *h_B = (float *)malloc(size);

    // Allocate the host output vector C
    float *h_C = (float *)malloc(size);

    // Verify that allocations succeeded
    if (h_A == NULL || h_B == NULL || h_C == NULL)
    {
        fprintf(stderr, "Failed to allocate host vectors!\n");
        exit(EXIT_FAILURE);
    }

    // Initialize the host input vectors
    for (int i = 0; i < numElements; ++i)
    {
        h_A[i] = rand()/(float)RAND_MAX;
        h_B[i] = rand()/(float)RAND_MAX;
    }

    // Allocate the device input vector A
    float *d_A = NULL;
    err = cudaMalloc((void **)&d_A, size);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to allocate device vector A (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Allocate the device input vector B
    float *d_B = NULL;
    err = cudaMalloc((void **)&d_B, size);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to allocate device vector B (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Allocate the device output vector C
    float *d_C = NULL;
    err = cudaMalloc((void **)&d_C, size);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to allocate device vector C (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Copy the host input vectors A and B in host memory to the device input vectors in
    // device memory
    printf("Copy input data from the host memory to the CUDA device\n");
    err = cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to copy vector A from host to device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    err = cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to copy vector B from host to device (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Launch the Vector Add CUDA Kernel
    int threadsPerBlock = 256;
    int blocksPerGrid =(numElements + threadsPerBlock - 1) / threadsPerBlock;
    printf("CUDA kernel launch with %d blocks of %d threads\n", blocksPerGrid, threadsPerBlock);
    vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, numElements);
    err = cudaGetLastError();

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to launch vectorAdd kernel (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Copy the device result vector in device memory to the host result vector
    // in host memory.
    printf("Copy output data from the CUDA device to the host memory\n");
    err = cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to copy vector C from device to host (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Verify that the result vector is correct
    for (int i = 0; i < numElements; ++i)
    {
        if (fabs(h_A[i] + h_B[i] - h_C[i]) > 1e-5)
        {
            fprintf(stderr, "Result verification failed at element %d!\n", i);
            exit(EXIT_FAILURE);
        }
    }

    printf("Test PASSED\n");

    // Free device global memory
    err = cudaFree(d_A);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to free device vector A (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    err = cudaFree(d_B);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to free device vector B (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    err = cudaFree(d_C);

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to free device vector C (error code %s)!\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    // Free host memory
    free(h_A);
    free(h_B);
    free(h_C);

    // Reset the device and exit
    // cudaDeviceReset causes the driver to clean up all state. While
    // not mandatory in normal operation, it is good practice.  It is also
    // needed to ensure correct operation when the application is being
    // profiled. Calling cudaDeviceReset causes all profile data to be
    // flushed before the application exits
    err = cudaDeviceReset();

    if (err != cudaSuccess)
    {
        fprintf(stderr, "Failed to deinitialize the device! error=%s\n", cudaGetErrorString(err));
        exit(EXIT_FAILURE);
    }

    printf("Done\n");
    return 0;
}

'File written in /content/src/helloCUDA.cu'

In [ ]:
!nvcc /content/src/helloCUDA.cu -o /content/src/helloCUDA

In [ ]:
!/content/src/helloCUDA

50000
[Vector addition of 50000 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 196 blocks of 256 threads
Copy output data from the CUDA device to the host memory
Test PASSED
Done


In [ ]:
%%cuda --name helloCUDA_input.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void vectorAdd(const float *A, const float *B, float *C, int numElements)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;

    if (i < numElements)
    {
        C[i] = A[i] + B[i];
    }
}

int main(void)
{
    // Error code to check return values for CUDA calls
    cudaError_t err = cudaSuccess;

    // Print the vector length to be used, and compute its size
    int numElements = 0;
    int t;
    scanf("%d",&t);
    while(t--)
    {
        scanf("%d",&numElements);

        size_t size = numElements * sizeof(float);
        printf("[Vector addition of %d elements]\n", numElements);

        // Allocate the host input vector A
        float *h_A = (float *)malloc(size);

        // Allocate the host input vector B
        float *h_B = (float *)malloc(size);

        // Allocate the host output vector C
        float *h_C = (float *)malloc(size);

        // Verify that allocations succeeded
        if (h_A == NULL || h_B == NULL || h_C == NULL)
        {
            fprintf(stderr, "Failed to allocate host vectors!\n");
            exit(EXIT_FAILURE);
        }

        // Initialize the host input vectors
        for (int i = 0; i < numElements; ++i)
        {
            h_A[i] = rand()/(float)RAND_MAX;
            h_B[i] = rand()/(float)RAND_MAX;
        }

        // Allocate the device input vector A
        float *d_A = NULL;
        err = cudaMalloc((void **)&d_A, size);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to allocate device vector A (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Allocate the device input vector B
        float *d_B = NULL;
        err = cudaMalloc((void **)&d_B, size);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to allocate device vector B (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Allocate the device output vector C
        float *d_C = NULL;
        err = cudaMalloc((void **)&d_C, size);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to allocate device vector C (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Copy the host input vectors A and B in host memory to the device input vectors in
        // device memory
        printf("Copy input data from the host memory to the CUDA device\n");
        err = cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to copy vector A from host to device (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to copy vector B from host to device (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Launch the Vector Add CUDA Kernel
        int threadsPerBlock = 256;
        int blocksPerGrid =(numElements + threadsPerBlock - 1) / threadsPerBlock;
        printf("CUDA kernel launch with %d blocks of %d threads\n", blocksPerGrid, threadsPerBlock);
        vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, numElements);
        err = cudaGetLastError();

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to launch vectorAdd kernel (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Copy the device result vector in device memory to the host result vector
        // in host memory.
        printf("Copy output data from the CUDA device to the host memory\n");
        err = cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to copy vector C from device to host (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Verify that the result vector is correct
        for (int i = 0; i < numElements; ++i)
        {
            if (fabs(h_A[i] + h_B[i] - h_C[i]) > 1e-5)
            {
                fprintf(stderr, "Result verification failed at element %d!\n", i);
                exit(EXIT_FAILURE);
            }
        }

        printf("Test PASSED\n");

        // Free device global memory
        err = cudaFree(d_A);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to free device vector A (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaFree(d_B);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to free device vector B (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaFree(d_C);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to free device vector C (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Free host memory
        free(h_A);
        free(h_B);
        free(h_C);

        // Reset the device and exit
        // cudaDeviceReset causes the driver to clean up all state. While
        // not mandatory in normal operation, it is good practice.  It is also
        // needed to ensure correct operation when the application is being
        // profiled. Calling cudaDeviceReset causes all profile data to be
        // flushed before the application exits
        err = cudaDeviceReset();

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to deinitialize the device! error=%s\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        printf("Done\n");
    }
    return 0;
}

'File written in /content/src/helloCUDA_input.cu'

In [ ]:
!nvcc /content/src/helloCUDA_input.cu -o helloCUDA_input

In [ ]:
!/content/helloCUDA_input < /content/input.txt


[Vector addition of 10000 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 40 blocks of 256 threads
Copy output data from the CUDA device to the host memory
Test PASSED
Done
[Vector addition of 20000 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 79 blocks of 256 threads
Copy output data from the CUDA device to the host memory
Test PASSED
Done
[Vector addition of 30000 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 118 blocks of 256 threads
Copy output data from the CUDA device to the host memory
Test PASSED
Done
[Vector addition of 40000 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 157 blocks of 256 threads
Copy output data from the CUDA device to the host memory
Test PASSED
Done
[Vector addition of 50000 elements]
Copy input data from the host memory to the CUDA device
CUDA kernel launch with 196 blocks of 256 thre

In [ ]:
%%cuda --name helloCUDA_input_test.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void vectorAdd(const float *A, const float *B, float *C, int numElements)
{
    int i = blockDim.x * blockIdx.x + threadIdx.x;

    if (i < numElements)
    {
        C[i] = A[i] + B[i];
    }
}

int main(void)
{
    // Error code to check return values for CUDA calls
    cudaError_t err = cudaSuccess;

    // Print the vector length to be used, and compute its size
    int numElements = 0;
    int t;
    scanf("%d",&t);
    while(t--)
    {
        scanf("%d",&numElements);

        size_t size = numElements * sizeof(float);
        //printf("[Vector addition of %d elements]\n", numElements);

        // Allocate the host input vector A
        float *h_A = (float *)malloc(size);

        // Allocate the host input vector B
        float *h_B = (float *)malloc(size);

        // Allocate the host output vector C
        float *h_C = (float *)malloc(size);

        // Verify that allocations succeeded
        if (h_A == NULL || h_B == NULL || h_C == NULL)
        {
            fprintf(stderr, "Failed to allocate host vectors!\n");
            exit(EXIT_FAILURE);
        }

        // Initialize the host input vectors
        for (int i = 0; i < numElements; ++i)
        {
            scanf("%f",&h_A[i]);
         
        }
        for (int i = 0; i < numElements; ++i)
        {
         
            scanf("%f",&h_B[i]);
        }

        
        // Allocate the device input vector A
        float *d_A = NULL;
        err = cudaMalloc((void **)&d_A, size);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to allocate device vector A (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Allocate the device input vector B
        float *d_B = NULL;
        err = cudaMalloc((void **)&d_B, size);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to allocate device vector B (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Allocate the device output vector C
        float *d_C = NULL;
        err = cudaMalloc((void **)&d_C, size);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to allocate device vector C (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Copy the host input vectors A and B in host memory to the device input vectors in
        // device memory
       // printf("Copy input data from the host memory to the CUDA device\n");
        err = cudaMemcpy(d_A, h_A, size, cudaMemcpyHostToDevice);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to copy vector A from host to device (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaMemcpy(d_B, h_B, size, cudaMemcpyHostToDevice);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to copy vector B from host to device (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Launch the Vector Add CUDA Kernel
        int threadsPerBlock = 256;
        int blocksPerGrid =(numElements + threadsPerBlock - 1) / threadsPerBlock;
        //printf("CUDA kernel launch with %d blocks of %d threads\n", blocksPerGrid, threadsPerBlock);
        vectorAdd<<<blocksPerGrid, threadsPerBlock>>>(d_A, d_B, d_C, numElements);
        err = cudaGetLastError();

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to launch vectorAdd kernel (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Copy the device result vector in device memory to the host result vector
        // in host memory.
       // printf("Copy output data from the CUDA device to the host memory\n");
        err = cudaMemcpy(h_C, d_C, size, cudaMemcpyDeviceToHost);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to copy vector C from device to host (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        // Verify that the result vector is correct
        for (int i = 0; i < numElements; ++i)
        {
            if (fabs(h_A[i] + h_B[i] - h_C[i]) > 1e-5)
            {
                fprintf(stderr, "Result verification failed at element %d!\n", i);
                exit(EXIT_FAILURE);
            }
        }

       // printf("Test PASSED\n");

        // Free device global memory
        err = cudaFree(d_A);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to free device vector A (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaFree(d_B);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to free device vector B (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

        err = cudaFree(d_C);

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to free device vector C (error code %s)!\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }

       

        // Reset the device and exit
        // cudaDeviceReset causes the driver to clean up all state. While
        // not mandatory in normal operation, it is good practice.  It is also
        // needed to ensure correct operation when the application is being
        // profiled. Calling cudaDeviceReset causes all profile data to be
        // flushed before the application exits
        err = cudaDeviceReset();

        if (err != cudaSuccess)
        {
            fprintf(stderr, "Failed to deinitialize the device! error=%s\n", cudaGetErrorString(err));
            exit(EXIT_FAILURE);
        }
        for (int i = 0; i < numElements; ++i)
        {
            printf("%.2f ",h_C[i]);
        }       
      // Free host memory
        free(h_A);
        free(h_B);
        free(h_C);
        printf("\n");
        //printf("Done\n");
    
    }
    return 0;
}

'File written in /content/src/helloCUDA_input_test.cu'

In [ ]:
!nvcc /content/src/helloCUDA_input_test.cu -o helloCUDA_input_test

In [ ]:
!/content/helloCUDA_input_test < /content/input_example.txt


1.23 1.58 1.11 1.11 0.83 1.11 0.87 1.87 1.36 0.75 0.26 0.94 0.56 0.24 1.22 1.35 0.91 1.16 1.46 1.06 1.30 1.29 0.63 1.73 1.02 0.62 0.85 1.24 0.08 0.52 1.21 1.75 0.81 1.14 1.18 0.57 1.37 1.65 1.02 0.99 0.86 1.32 1.16 1.12 1.04 1.55 1.52 1.36 1.21 1.59 0.70 1.87 1.03 1.07 0.90 1.10 0.68 0.47 0.98 1.08 0.23 1.26 1.92 1.06 1.12 0.52 0.82 0.88 0.92 0.91 1.02 0.57 0.42 1.53 1.63 1.40 0.22 0.60 0.27 1.28 1.33 0.21 1.20 1.02 1.05 0.94 0.92 0.77 0.55 1.73 0.91 1.03 0.79 1.29 1.29 1.46 1.38 0.79 1.76 1.70 1.72 1.88 1.17 0.99 0.97 0.90 0.86 1.02 0.56 0.70 0.87 1.48 0.83 0.83 1.06 1.39 1.19 0.43 0.94 1.06 1.16 0.78 0.89 0.66 1.11 1.31 0.53 1.24 1.06 0.94 1.60 0.91 0.62 1.14 0.78 0.47 1.17 0.96 1.07 0.52 0.98 1.16 0.63 1.88 1.46 0.81 1.15 1.86 1.06 1.34 1.20 1.78 1.08 0.43 1.35 1.31 0.79 0.86 0.87 1.09 1.56 0.40 0.62 0.50 1.25 1.53 1.66 0.83 1.05 0.87 0.22 0.76 0.65 0.42 1.36 0.85 1.42 0.65 1.51 0.73 0.99 1.22 0.57 1.37 1.32 1.04 0.42 1.55 0.40 1.59 0.50 1.70 0.68 0.96 0.48 0.96 0.94 0.10 1.24 1.13 

In [4]:
%%cuda --name GPU_features.cu
#include<cuda.h>
#include<stdio.h>
void printDevProp(cudaDeviceProp);
int main()
{
    int devCount;
    cudaGetDeviceCount(&devCount);
    for(int i=0;i<devCount;++i)
    {
        cudaDeviceProp devp;
        cudaGetDeviceProperties(&devp,i);
        printDevProp(devp);
    }
    return 0;
}
void printDevProp(cudaDeviceProp devProp)
{
    printf(" Major revision number : %d\n",devProp.major);
    printf(" Minor revision number : %d\n",devProp.minor);
    printf(" Name : %s\n",devProp.name);
    printf(" Total global memory : %lu\n",devProp.totalGlobalMem);
    printf(" Total shared memory per block : %lu\n",devProp.sharedMemPerBlock);
    printf(" Total registers per block : %d\n", devProp.regsPerBlock);
    printf(" Warp size : %d\n",devProp.warpSize);
    printf(" Maximum memory pitch : %lu\n",devProp.memPitch);
    printf(" Maximum threads per block : %d\n",devProp.maxThreadsPerBlock);
    for(int i=0;i<3;++i)
        printf (" Maximum dimension %d of block : %d\n",i,devProp.maxThreadsDim[i]);
    for(int i=0;i<3;++i)
        printf (" Maximum dimension %d of grid : %d\n",i,devProp.maxGridSize[i]);
    printf(" Clock rate : %d\n",devProp.clockRate);
    printf(" Total constant memory : %lu\n",devProp.totalConstMem);
    printf(" Texture alignment : %lu\n",devProp.textureAlignment);
    printf(" Concurrent copy and execution : %s\n",(devProp.deviceOverlap?"Yes":"No"));
    printf(" Number of multiprocessors : %d\n",devProp.multiProcessorCount);
    return;
}

'File written in /content/src/GPU_features.cu'

In [5]:
!nvcc /content/src/GPU_features.cu -o GPU_features

In [6]:
!/content/GPU_features

 Major revision number : 7
 Minor revision number : 5
 Name : Tesla T4
 Total global memory : 15843721216
 Total shared memory per block : 49152
 Total registers per block : 65536
 Warp size : 32
 Maximum memory pitch : 2147483647
 Maximum threads per block : 1024
 Maximum dimension 0 of block : 1024
 Maximum dimension 1 of block : 1024
 Maximum dimension 2 of block : 64
 Maximum dimension 0 of grid : 2147483647
 Maximum dimension 1 of grid : 65535
 Maximum dimension 2 of grid : 65535
 Clock rate : 1590000
 Total constant memory : 65536
 Texture alignment : 512
 Concurrent copy and execution : Yes
 Number of multiprocessors : 40


In [7]:
!nvidia-smi -q


==============NVSMI LOG==============

Timestamp                                 : Tue Mar  9 21:02:16 2021
Driver Version                            : 460.32.03
CUDA Version                              : 11.2

Attached GPUs                             : 1
GPU 00000000:00:04.0
    Product Name                          : Tesla T4
    Product Brand                         : Tesla
    Display Mode                          : Enabled
    Display Active                        : Disabled
    Persistence Mode                      : Disabled
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial Number                         : 1561920022586
    GPU UUID                              : GPU-09bf90f4-c633-606e-e9b2-0a7f0